<a href="https://colab.research.google.com/github/leninvarma11/Homework-1-Lenin-Varma-Nallapu-700076011/blob/main/Homework-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Question
 1)Write a regex to find U.S. ZIP codes (disjunction + token boundaries) Match 12345 or 12345-6789 or 12345 6789 (hyphen or space allowed for the +4 part). Make sure you only match whole tokens (not inside longer strings).

Ans:\b\d{5}(?:[ -]\d{4})?\b

2)Negation in disjunction (word start rules) Find all words that do not start with a capital letter. Words may include internal apostrophes/hyphens like don’t, state-of-the-art.

Ans: \b(?![A-Z])[a-z]+(?:['’-][a-z]+)*\b

3)Convenient aliases (numbers, a bit richer) Extract all numbers that may have: optional sign (+/-), optional thousands separators (commas), optional decimal part, optional scientific notation (e.g., 1.23e-4).

Ans: \b[+-]?(?:(?:\d{1,3}(?:,\d{3})+|\d+)(?:.\d+)?|.\d+)(?:[eE][+-]?\d+)?\b

4)More disjunction (spelling variants) Match any spelling of “email”: email, e-mail, or e mail. Accept either a space or a hyphen (including en‐dash –) between e and mail, and be case-insensitive.

Ans: (?i)\bE(?:[- ]?)mail\b With en- dash : (?i)\bE(?:\s|[-\u2013])?mail\b

5 )Wildcards, optionality, repetition (with punctuation) Match the interjection go, goo, gooo, … (one or more o), as a word, and allow an optional trailing punctuation mark ! . , ? (e.g., gooo!).

ANS: \bgo+\b[!.,?]?

6)Anchors (line/sentence end with quotes)
Match lines that end with a question mark possibly followed only by closing quotes/brackets like ")”’] and spaces.

Ans :(?m)^[^\n]?[)"]\u201D\u2019\s]$



In [3]:
#2 Question

import spacy

# === 1. Paragraph ===
paragraph = "Yesterday, I went to New York City. It’s a busy place, full of energy! However, I didn’t stay long."

# === 1a. Naïve space-based tokenization ===
naive_tokens = paragraph.split()

# === 1b. Manual correction (punctuation, clitics, MWEs) ===
manual_tokens = [
    'Yesterday', ',', 'I', 'went', 'to', 'New York City', '.',
    'It', '’s', 'a', 'busy', 'place', ',', 'full', 'of', 'energy', '!',
    'However', ',', 'I', 'did', 'n’t', 'stay', 'long', '.'
]

# === 2. Tool-based tokenization (spaCy) ===
nlp = spacy.load("en_core_web_sm")
doc = nlp(paragraph)
spacy_tokens = [t.text for t in doc]

# === 3. MWEs ===
mwes = [
    "New York City - place name, should be one token.",
    "social media - fixed phrase with non-compositional meaning.",
    "kick the bucket - idiom meaning 'to die'."
]

# === 4. Reflection text ===
reflection = """The hardest part of tokenization was handling contractions and MWEs.
For example, deciding whether to split 'didn’t' depends on whether the task requires grammatical structure or surface forms.
Compared with English, some languages are harder because of rich morphology and agglutination, where one word encodes multiple morphemes.
Punctuation is relatively easy but still introduces edge cases, like apostrophes.
Morphology and MWEs make tokenization more difficult because they require semantic knowledge beyond surface rules.
Overall, tokenization is a balance between linguistic accuracy and tool consistency.
"""

# === Print Outputs ===
print("=== 1a. Naïve space-based tokens ===")
print(naive_tokens, "\n")

print("=== 1b. Manual corrected tokens ===")
print(manual_tokens, "\n")

print("=== 2. spaCy tokens ===")
print(spacy_tokens, "\n")

print("=== Differences (Manual vs spaCy) ===")
print("In manual but not spaCy:", set(manual_tokens) - set(spacy_tokens))
print("In spaCy but not manual:", set(spacy_tokens) - set(manual_tokens), "\n")

print("=== 3. Multiword Expressions (MWEs) ===")
for m in mwes:
    print("-", m)
print()

print("=== 4. Reflection ===")
print(reflection)


=== 1a. Naïve space-based tokens ===
['Yesterday,', 'I', 'went', 'to', 'New', 'York', 'City.', 'It’s', 'a', 'busy', 'place,', 'full', 'of', 'energy!', 'However,', 'I', 'didn’t', 'stay', 'long.'] 

=== 1b. Manual corrected tokens ===
['Yesterday', ',', 'I', 'went', 'to', 'New York City', '.', 'It', '’s', 'a', 'busy', 'place', ',', 'full', 'of', 'energy', '!', 'However', ',', 'I', 'did', 'n’t', 'stay', 'long', '.'] 

=== 2. spaCy tokens ===
['Yesterday', ',', 'I', 'went', 'to', 'New', 'York', 'City', '.', 'It', '’s', 'a', 'busy', 'place', ',', 'full', 'of', 'energy', '!', 'However', ',', 'I', 'did', 'n’t', 'stay', 'long', '.'] 

=== Differences (Manual vs spaCy) ===
In manual but not spaCy: {'New York City'}
In spaCy but not manual: {'City', 'York', 'New'} 

=== 3. Multiword Expressions (MWEs) ===
- New York City - place name, should be one token.
- social media - fixed phrase with non-compositional meaning.
- kick the bucket - idiom meaning 'to die'.

=== 4. Reflection ===
The hardest p

In [2]:
#3 Question
from collections import Counter

def add_eow(word):
    return " ".join(list(word)) + " _"

def get_stats(tokens):
    pairs = Counter()
    for word in tokens:
        for i in range(len(word)-1):
            pairs[(word[i], word[i+1])] += 1
    return pairs

def merge_pair(tokens, pair):
    new_tokens = []
    bigram = " ".join(pair)
    replacement = "".join(pair)
    for word in tokens:
        joined = " ".join(word)
        merged = joined.replace(bigram, replacement)
        new_tokens.append(merged.split())
    return new_tokens

def segment(word, merges):
    chars = list(word) + ["_"]
    for merge in merges:
        i = 0
        while i < len(chars)-1:
            if (chars[i], chars[i+1]) == merge:
                chars[i:i+2] = ["".join(merge)]
            else:
                i += 1
    return chars

# ============================================================
# 3.1 Manual BPE on Toy Corpus
# ============================================================
print("\n=== 3.1 Manual BPE (Toy Corpus) ===")
toy_corpus = ["low","low","low","low","low","lowest","lowest",
              "newer","newer","newer","newer","newer","newer",
              "wider","wider","wider","new","new"]

corpus_tokens = [add_eow(w).split() for w in toy_corpus]

# Step 1: compute stats
pairs = get_stats(corpus_tokens)
print("Bigram counts (top 5):", pairs.most_common(5))

# First 3 merges manually
merges_manual = []
for step in range(3):
    pairs = get_stats(corpus_tokens)
    best = pairs.most_common(1)[0][0]
    merges_manual.append(best)
    corpus_tokens = merge_pair(corpus_tokens, best)
    print(f"Step {step+1}: merge {best}, new vocab token: {''.join(best)}")

print("First 3 merges:", merges_manual)

# ============================================================
# 3.2 Mini BPE Learner (Toy Corpus again)
# ============================================================
print("\n=== 3.2 Mini BPE Learner (Toy Corpus) ===")
corpus_tokens = [add_eow(w).split() for w in toy_corpus]
merges = []
for i in range(10):
    pairs = get_stats(corpus_tokens)
    if not pairs: break
    best = pairs.most_common(1)[0][0]
    merges.append(best)
    corpus_tokens = merge_pair(corpus_tokens, best)
    print(f"Step {i+1}: merge {best}, vocab size: {len(set(tok for word in corpus_tokens for tok in word))}")

# Segment sample words
test_words = ["new", "newer", "lowest", "widest", "newestest"]
print("\nSegmentation examples:")
for w in test_words:
    print(w, "->", segment(w, merges))

# ============================================================
# 3.3 Training BPE on Paragraph
# ============================================================
print("\n=== 3.3 BPE on Paragraph ===")
paragraph = """Artificial intelligence is transforming industries.
Machine learning models are trained on vast amounts of data.
They can recognize images, translate text, and even generate new content.
However, ethical concerns remain. Fairness and bias are critical issues.
Researchers continue to explore solutions."""

# Prepare corpus
words = paragraph.lower().replace(".", "").split()
corpus_para = [add_eow(w).split() for w in words]

# Train 30 merges
tokens = corpus_para
merges_para = []
for i in range(30):
    pairs = get_stats(tokens)
    if not pairs: break
    best = pairs.most_common(1)[0][0]
    merges_para.append(best)
    tokens = merge_pair(tokens, best)

print("Top 5 merges:", merges_para[:5])

# 5 longest tokens
vocab = set(tok for word in tokens for tok in word)
longest = sorted(vocab, key=len, reverse=True)[:5]
print("Longest 5 tokens:", longest)

# Segment 5 words
test_words_para = ["artificial", "researchers", "bias", "transforming", "solutions"]
print("\nSegmentation (Paragraph words):")
for w in test_words_para:
    print(w, "->", segment(w, merges_para))



=== 3.1 Manual BPE (Toy Corpus) ===
Bigram counts (top 5): [(('e', 'r'), 9), (('r', '_'), 9), (('w', 'e'), 8), (('n', 'e'), 8), (('e', 'w'), 8)]
Step 1: merge ('e', 'r'), new vocab token: er
Step 2: merge ('er', '_'), new vocab token: er_
Step 3: merge ('n', 'e'), new vocab token: ne
First 3 merges: [('e', 'r'), ('er', '_'), ('n', 'e')]

=== 3.2 Mini BPE Learner (Toy Corpus) ===
Step 1: merge ('e', 'r'), vocab size: 11
Step 2: merge ('er', '_'), vocab size: 11
Step 3: merge ('n', 'e'), vocab size: 11
Step 4: merge ('ne', 'w'), vocab size: 11
Step 5: merge ('l', 'o'), vocab size: 10
Step 6: merge ('lo', 'w'), vocab size: 10
Step 7: merge ('new', 'er_'), vocab size: 11
Step 8: merge ('low', '_'), vocab size: 12
Step 9: merge ('w', 'i'), vocab size: 11
Step 10: merge ('wi', 'd'), vocab size: 10

Segmentation examples:
new -> ['new', '_']
newer -> ['newer_']
lowest -> ['low', 'e', 's', 't', '_']
widest -> ['wid', 'e', 's', 't', '_']
newestest -> ['new', 'e', 's', 't', 'e', 's', 't', '_']


#4 Question Word Pair:
Sunday → Saturday
Tasks:

#1. Find the minimum edit distance between Sunday and Saturday under both models:

Model A (Sub = 1, Ins = 1, Del = 1)

Model B (Sub = 2, Ins = 1, Del = 1)

1) Minimum edit distance

Model A (Sub=1, Ins=1, Del=1): Distance = 3

Model B (Sub=2, Ins=1, Del=1): Distance = 4


#2 . One valid edit sequence
Model A (cost 3)

Insert a after S → Saunday

Insert t after Sa → Satunday

Substitute n → r → Saturday

Model B (cost 4)

Insert a after S → Saunday

Insert t after Sa → Satunday

Delete n → Satuday

Insert r before d → Saturday

#3. Reflection (4–5 sentences)

The two models gave different distances: Model A = 3, Model B = 4. In Model A, substitution was cheap and useful, so it was chosen for the mismatch.

In Model B, substitution was expensive, so the algorithm preferred a delete + insert instead. The choice of model matters: for spell-checking, substitutions should be cheap since typos often involve wrong letters.

For DNA alignment, substitutions (mutations) may be more costly or biologically significant, so making substitutions expensive produces more realistic alignments.
     